<a href="https://colab.research.google.com/github/Deepadarshni/SentimentAnalysis-of-UserReviews-with-EmojiAnalysis/blob/main/Sentiment_Analysis_of_User_reviews_with_emoji_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install emoji
!pip install emoji==1.7.0
import pandas as pd
import numpy as np
import emoji
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense
from scipy.stats import pearsonr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 358.9/358.9 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171031 sha256=5c36ee3da245bfc2b18513d1fdca893d8eaef1323f50b0efae21851a48bd344b
  Stored in directory: /root/.cache/pip/wheels/31/8a/8c/315c9e5d7773f74b33d5ed33f075b49c6eaeb7cedbb86e2cf8
Successfully built emoji
  Attempting uninstall: emoji
    Found existing installation: emoji 2.8.0
    Uninstalling emoji-2.8.0:
      Successfully uninstalled emoji-2.8.0


In [ ]:
df = pd.read_csv('Customer Reviews1.csv')
print(df)

                                     text     label
0                    Amazing features 😍 🤩  positive
1                 I love this product ! 😊  positive
2                       worst quality 😞 😞  negative
3        Not satisfied with the product 👎  negative
4         I am happy with the feature 🤩 🤑  positive
5               It's an average product 🤔   neutral
6   Terrible quality,regret buying it 👎 👎  negative
7                     could be better 🤔 😐   neutral
8       I am satisfied with my purchase 😊  positive
9                Not recommended to buy 🥲  negative
10                Absolutely love it! 😊 💯  positive
11      Very disappointing experience 🥲 🥲  negative
12                  Worst purchase ever 😞  negative
13                  It's okay,not great 🤔   neutral
14         Excellent customer service 😍 😍  positive
15       The design is sleek and modern 🤩  positive
16         Very user-friendly product 👏 👏  positive
17  Disappointed with the performance 🥲 🥲  negative
18          

In [ ]:
# Analyze sentiment of emojis during preprocessing
def preprocess_and_analyze_sentiment(text):
    text = emoji.demojize(text)
    text = text.lower()

    # Analyze sentiment of emojis
    emojis = [char for char in text if char in emoji.demojize(text)]
    # Map emoji names to sentiments
    emoji_sentiment_mapping = {
        ':smiling_face_with_heart-eyes:': 'positive', ':thumbs_up:': 'positive', ':clap:': 'positive', ':star_struck:': 'positive',
        ':disappointed:': 'negative', ':thumbs_down:': 'negative', ':sob:': 'negative', ':angry:': 'negative',':pensive_face:':'negative',
        ':smile:': 'positive', ':thinking_face:': 'neutral', ':money_mouth_face:': 'positive',
        ':face_with_rolling_eyes:': 'negative', ':rocket:': 'positive', ':poop:': 'negative',
        ':shopping_bags:': 'positive', ':100:': 'positive',':crying_face:':'negative'
    }
    sentiment = [emoji_sentiment_mapping.get(char, 'neutral') for char in emojis]

    return text, sentiment


In [ ]:
# Apply preprocessing and sentiment analysis
df[['text', 'emoji_sentiment']] = df['text'].apply(preprocess_and_analyze_sentiment).apply(pd.Series)
print(df[['text', 'emoji_sentiment']])

                                                 text  \
0   amazing features :smiling_face_with_heart-eyes...   
1   i love this product ! :smiling_face_with_smili...   
2   worst quality :disappointed_face: :disappointe...   
3        not satisfied with the product :thumbs_down:   
4   i am happy with the feature :star-struck: :mon...   
5             it's an average product :thinking_face:   
6   terrible quality,regret buying it :thumbs_down...   
7      could be better :thinking_face: :neutral_face:   
8   i am satisfied with my purchase :smiling_face_...   
9     not recommended to buy :smiling_face_with_tear:   
10  absolutely love it! :smiling_face_with_smiling...   
11  very disappointing experience :smiling_face_wi...   
12            worst purchase ever :disappointed_face:   
13                it's okay,not great :thinking_face:   
14  excellent customer service :smiling_face_with_...   
15       the design is sleek and modern :star-struck:   
16  very user-friendly product 

In [ ]:
# Feature extraction using TF-IDF and Skip N-gram
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2), max_features=1000)
X_tfidf = tfidf_vectorizer.fit_transform(df['text']).toarray()

In [ ]:
# Merge TF-IDF features with sentiment of emojis
X_combined = np.concatenate([X_tfidf, np.array(df['emoji_sentiment'].apply(lambda x: 1 if 'positive' in x else (-1 if 'negative' in x else 0))).reshape(-1, 1)], axis=1)


In [ ]:
# Encode labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['label'].values)
print(y)

[2 2 0 0 2 1 0 1 2 0 2 0 0 1 2 2 2 0 2 0 1 2 2 2 0 0 0 0 0]


In [ ]:
# Pearson correlation coefficient for feature selection
correlations = [pearsonr(X_combined[:, i], y)[0] for i in range(X_combined.shape[1])]


/usr/local/lib/python3.10/dist-packages/scipy/stats/_stats_py.py:4781: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))


In [ ]:
# Select features based on correlation threshold (you can adjust this threshold)
selected_features = [i for i, corr in enumerate(correlations) if abs(corr) > 0.1]
X_selected = X_combined[:, selected_features]

In [ ]:
# Tokenize and pad sequences for RNN-LSTM
max_words_text = 1000
max_len_text = 20

tokenizer = Tokenizer(num_words=max_words_text, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower=True, split=' ')
tokenizer.fit_on_texts(df['text'].values)
X_text = tokenizer.texts_to_sequences(df['text'].values)
X_text = pad_sequences(X_text, maxlen=max_len_text)

In [ ]:
# Build the RNN-LSTM model
model = Sequential()
model.add(Embedding(input_dim=max_words_text, output_dim=50, input_length=max_len_text))
model.add(LSTM(units=100, activation='tanh'))
model.add(Dense(units=10, activation='relu'))
model.add(Dense(units=3, activation='softmax'))  # Output layer for three classes (positive, negative, neutral)

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Train-test split for RNN-LSTM
X_train_text, X_test_text, y_train, y_test = train_test_split(X_text, y, test_size=0.2, random_state=42)
X_train_selected, X_test_selected, _, _ = train_test_split(X_selected, y, test_size=0.2, random_state=42)

In [ ]:
# Train the model
model.fit(X_train_text, y_train, epochs=10, batch_size=32, validation_split=0.2)


Epoch 1/10
1/1 [==============================] - 3s 3s/step - loss: 1.0990 - accuracy: 0.2222 - val_loss: 1.0965 - val_accuracy: 0.4000
Epoch 2/10
1/1 [==============================] - 0s 71ms/step - loss: 1.0949 - accuracy: 0.3889 - val_loss: 1.0928 - val_accuracy: 0.4000
Epoch 3/10
1/1 [==============================] - 0s 96ms/step - loss: 1.0898 - accuracy: 0.3889 - val_loss: 1.0898 - val_accuracy: 0.4000
Epoch 4/10
1/1 [==============================] - 0s 60ms/step - loss: 1.0850 - accuracy: 0.5556 - val_loss: 1.0866 - val_accuracy: 0.4000
Epoch 5/10
1/1 [==============================] - 0s 76ms/step - loss: 1.0790 - accuracy: 0.5556 - val_loss: 1.0834 - val_accuracy: 0.4000
Epoch 6/10
1/1 [==============================] - 0s 76ms/step - loss: 1.0725 - accuracy: 0.5556 - val_loss: 1.0791 - val_accuracy: 0.4000
Epoch 7/10
1/1 [==============================] - 0s 76ms/step - loss: 1.0648 - accuracy: 0.5556 - val_loss: 1.0744 - val_accuracy: 0.4000
Epoch 8/10
1/1 [=============

In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test_text, y_test)
print(f"Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}")

1/1 [==============================] - 0s 32ms/step - loss: 0.9991 - accuracy: 0.8333
Test Loss: 0.9991, Test Accuracy: 0.8333


In [ ]:
# Predicting new data
new_texts = ["Happy with the purchase! 😍👍","Disappointing experience. 😔😢", "It's okay.🤔"]
new_texts_preprocessed = [preprocess_and_analyze_sentiment(text)[0] for text in new_texts]
print(new_texts_preprocessed)

['happy with the purchase! :smiling_face_with_heart-eyes::thumbs_up:', 'disappointing experience. :pensive_face::crying_face:', "it's okay.:thinking_face:"]


In [ ]:
# Tokenize and pad sequences for new text
X_new_text = tokenizer.texts_to_sequences(new_texts_preprocessed)
print(X_new_text)
X_new_text = pad_sequences(X_new_text, maxlen=max_len_text)
print(X_new_text)

[[45, 3, 5, 29, 2, 1, 3, 15, 6, 8], [60, 61, 1, 1], [25, 63, 14, 1]]
[[ 0  0  0  0  0  0  0  0  0  0 45  3  5 29  2  1  3 15  6  8]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 60 61  1  1]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 25 63 14  1]]


In [ ]:
 #Predict sentiments for new data
predictions = model.predict(X_new_text)
print(predictions)

1/1 [==============================] - 1s 583ms/step
[[0.3595397  0.26513383 0.37532648]
 [0.3539489  0.2721948  0.3738563 ]
 [0.34676448 0.28229824 0.37093732]]


In [ ]:
# Display predictions
for i, text in enumerate(new_texts):
    print(f"Text: {text}, Predicted Probability (Positive, Negative, Neutral): {predictions[i]}")

Text: Happy with the purchase! 😍👍, Predicted Probability (Positive, Negative, Neutral): [0.3595397  0.26513383 0.37532648]
Text: Disappointing experience. 😔😢, Predicted Probability (Positive, Negative, Neutral): [0.3539489 0.2721948 0.3738563]
Text: It's okay.🤔, Predicted Probability (Positive, Negative, Neutral): [0.34676448 0.28229824 0.37093732]


In [ ]:
# Convert probabilities to binary predictions
binary_predictions = np.argmax(predictions, axis=1)
print(binary_predictions)

[2 2 2]


In [ ]:
# Display binary predictions
for i, text in enumerate(new_texts):
    sentiment = 'Positive' if binary_predictions[i] == 0 else ('Negative' if binary_predictions[i] == 1 else 'Neutral')
    print(f"Text: {text}, Predicted Sentiment:")